In [ ]:
# Imports

import pandas as pd
from pandas import json_normalize
from SPARQLWrapper import SPARQLWrapper, JSON

# Setup

In [ ]:
SPOTIFY_ONTOLOGY = "https://www.dei.unipd.it/~martinelli/spotify/spotifyOntology#"
REPOSITORY_NAME = "eu"

SPARQL_ENDPOINT = "http://localhost:7200/repositories/" + REPOSITORY_NAME

sparql = SPARQLWrapper(SPARQL_ENDPOINT)


## Query utilities

In [ ]:
# Execute a SPARQL query and return a DataFrame
def executeQuery(query, maxRows=None):
    global sparql

    # Set the number of rows and columns to display
    pd.set_option("display.max_rows", maxRows, "display.max_columns", None)

    # Set the query
    sparql.setQuery("""
            prefix so: <https://www.dei.unipd.it/~martinelli/spotify/spotifyOntology#>
            prefix countries: <http://eulersharp.sourceforge.net/2003/03swap/countries#>
            prefix xsd: <http://www.w3.org/2001/XMLSchema#>

            {query}
        """.format(query=query))

    # Set the result format
    sparql.setReturnFormat(JSON)

    # Execute the query
    results = sparql.query().convert()

    # If it is an ask
    if "boolean" in results.keys():
        return results["boolean"], "ask"
    
    # If it is a select
    if "results" in results.keys():
        # Create a DataFrame from the returned JSON
        resultDF = json_normalize(results["results"]["bindings"])

        # Remove the datadtypes columns
        resultDF = resultDF[[col for col in resultDF.columns if not col.endswith(
            ".type") and not col.endswith(".datatype")]]

        # Remove .value from column names
        resultDF = resultDF.rename(columns=lambda col: col.replace(".value", ""))

        # Remove the spotify ontology prefix from uris (to get only the ID)
        resultDF = resultDF.applymap(lambda x: str(x).lstrip(SPOTIFY_ONTOLOGY))

        return resultDF, "select"
    
    return None


In [ ]:
def printResult(result, resultType):
    if resultType == "select":
        print("🗃️ [[RESULT]]\n")
        result

    if resultType == "ask":
        print("🎱 [[ANSWER]]\n")
        print("✔️ Yes" if result else "❌ No")


# Queries

In [ ]:
result, resultType = executeQuery("""
ASK WHERE{
    {
        SELECT (COUNT (DISTINCT ?track) as ?numSinglesAG) WHERE{
            ?track so:isPartOf ?album ;
             	   so:name ?trackName ;
    			   so:appearsIn ?appearance .
        	?album so:isTypeOf so:single;
                	so:name ?albumName .	
        	?appearance so:isPositionedIn ?chart .
        	?chart so:isReferredTo countries:ar ;
                   so:date ?date .
    		FILTER ((?date >= "2018-01-01"^^xsd:date && ?date < "2019-01-01"^^xsd:date))
		}
    }
    {
        SELECT (COUNT (DISTINCT ?track) as ?numSinglesIT) WHERE{
            ?track so:isPartOf ?album ;
             	   so:name ?trackName ;
    			   so:appearsIn ?appearance .
        	?album so:isTypeOf so:single;
                	so:name ?albumName .	
        	?appearance so:isPositionedIn ?chart .
        	?chart so:isReferredTo countries:it ;
                   so:date ?date .
    		FILTER ((?date >= "2018-01-01"^^xsd:date && ?date < "2019-01-01"^^xsd:date))
		}
    }
    FILTER (?numSinglesAG > ?numSinglesIT)
}
""")

printResult(result, resultType)
